In [ ]:
### Load Dependencies ###

import teradata
import pandas
import os
import config
import lockout

In [ ]:
print(str(pandas.datetime.now()))
print("SLINKYDOG SETUP - START")

In [ ]:
### Automation --- Program Lockout Check ###

# Used to prevent multiple instances from running

# Builtin default lockout file is py.lock
# lockout.lock_file = "py.lock"

In [ ]:
# Manual lockout override
# lockout.unlock(verbose=True)

In [ ]:
# Check for lockout
# Passing hault=True will stop the program
lockout.islocked(verbose=True, hault=True)

In [ ]:
# Enable lockout before continuing
lockout.lock(verbose=True)

In [ ]:
### Setup and open Teradata connection ###

os.system('echo -n "$TERADATA_LOGON_PASSWORD" | kinit "$TERADATA_LOGON_USER"');
udaExec = teradata.UdaExec(appName="python", version="1.0", logConsole=False,
                          odbcLibPath="/opt/app/ttu1510/teradata/client/15.10/odbc_64/lib/libodbc.so")
session = udaExec.connect(method="odbc", system="edwprod.dw.medcity.net", driver="Teradata")

### Create placeholders for all tables

In [ ]:
### Tables To Initiate ###

tables = [
    config.SNAPSHOT_STAGING,
    config.SNAPSHOT_CURRENT,
    config.SNAPSHOT_PREVIOUS,
    config.KEYFRAME_TABLE,
    config.INTRAFRAME_MEMO_STAGING,
    config.INTRAFRAME_STAGING,
    config.INTRAFRAME_TABLE,
    config.INDEX_TABLE
]

In [ ]:
### Clear any tables that exist ###
for table in tables:
    print(table)
    try:
        session.execute("DROP TABLE " + table + ";")
    except:
        print("ERROR")

In [ ]:
### Create placeholders for tables ###

print("Creating table placeholders...")

for table in tables:
    print(table)
    session.execute("CREATE TABLE " + table + " (placeholder int NOT NULL);")

### Build the Index Table

In [ ]:
### Build Index Table ###

print("Dropping INDEX_TABLE placeholder...")
query = "DROP TABLE " + config.INDEX_TABLE + ";"

session.execute(query)

print("Building empty INDEX_TABLE...")
query = "CREATE TABLE " + config.INDEX_TABLE + """
(
    INDEX_SEQ int,
    KEYFRAME_SEQ int,
    INTRAFRAME_SEQ int,
    FRAME_TIMESTAMP timestamp
);
"""
    
session.execute(query)

### Close Teradata Connection

In [ ]:
### Close Session ###
session.close()

print(str(pandas.datetime.now()))
print("Session Closed.")

lockout.unlock(verbose=True)

In [ ]:
print(str(pandas.datetime.now()))
print("SLINKYDOG SETUP - COMPLETE")